# You might have to open the raw data file in libra office and merge the dilimiters

In [2]:
import pandas as pd
import numpy as np
#import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

# Import curve fitting package from scipy
from sklearn.linear_model import LinearRegression

# import os
import os, sys, shutil

##################################################################
# define error handler
class UnAcceptedValueError(Exception):   
    def __init__(self, data):    
        self.data = data
    def __str__(self):
        return repr(self.data)

####################################################################
# gets all items in directory
items = os.listdir(".")


# lists all .csv
csv_list = []
for names in items:
    if names.endswith(".csv"):
        csv_list.append(names)
print(csv_list)
try:
    if(len(csv_list) > 1):
        raise UnAcceptedValueError("More than 1x .CSV file in the directory");
except UnAcceptedValueError as e:
    print ("Received error:", e.data)
    # kills the process
    quit()
##########################################################################################
#import dataset as dataframe

raw_data = pd.read_csv(csv_list[0], sep=',', header=None)


['154253_230202_OT2_BRADFORD.csv', 'Consensus_concentration.csv']
Received error: More than 1x .CSV file in the directory


In [3]:
# get the row index of the cell that contains the string "Corrected [Bradford:595]"
row_index = raw_data.index[raw_data.loc[:,0]=="Corrected [Bradford:595]"][0]

corrected_data = raw_data.iloc[row_index+2: row_index + 10, 1 : 13].reset_index(drop=True)
print(corrected_data)

      1      2      3      4      5                         6      7      8   \
0  1.422  1.420  1.459  0.595  0.601                     0.607  0.649  0.595   
1  1.042  1.076  1.077  1.383  1.373                     1.386  1.377  1.374   
2  0.778  0.801  0.800  1.720  1.705                      1.73  1.721  1.735   
3  1.929  1.923  1.920  1.937  1.945  Corrected [Bradford:595]    NaN    NaN   
4  2.137  2.139  2.148  2.143  2.144  Corrected [Bradford:595]    NaN    NaN   
5  2.319  2.306  2.307  2.306  2.378  Corrected [Bradford:595]    NaN    NaN   
6  2.489  2.512  2.499  2.495  2.462  Corrected [Bradford:595]    NaN    NaN   
7  2.567  2.564  2.552  2.555  2.559  Corrected [Bradford:595]    NaN    NaN   

                         9   10  11  12  
0  Corrected [Bradford:595] NaN NaN NaN  
1  Corrected [Bradford:595] NaN NaN NaN  
2  Corrected [Bradford:595] NaN NaN NaN  
3                       NaN NaN NaN NaN  
4                       NaN NaN NaN NaN  
5                       NaN

In [4]:
# hot fix to deal with the parser not being able to parse "blank wells".
# remove at a later date

calibrants = corrected_data.iloc[:,0:5]
print(calibrants)
print(type(calibrants.iloc[0,0]))

       1      2      3      4      5
0  1.422  1.420  1.459  0.595  0.601
1  1.042  1.076  1.077  1.383  1.373
2  0.778  0.801  0.800  1.720  1.705
3  1.929  1.923  1.920  1.937  1.945
4  2.137  2.139  2.148  2.143  2.144
5  2.319  2.306  2.307  2.306  2.378
6  2.489  2.512  2.499  2.495  2.462
7  2.567  2.564  2.552  2.555  2.559
<class 'numpy.float64'>


In [5]:
new_low = calibrants.iloc[0,0:3].copy()
new_high = calibrants.iloc[2,0:3].copy()


calibrants.iloc[0,0:3] = new_high
calibrants.iloc[2,0:3] = new_low

print(calibrants)

       1      2      3      4      5
0  0.778  0.801  0.800  0.595  0.601
1  1.042  1.076  1.077  1.383  1.373
2  1.422  1.420  1.459  1.720  1.705
3  1.929  1.923  1.920  1.937  1.945
4  2.137  2.139  2.148  2.143  2.144
5  2.319  2.306  2.307  2.306  2.378
6  2.489  2.512  2.499  2.495  2.462
7  2.567  2.564  2.552  2.555  2.559


In [1]:
# reshape

calibrants_np = calibrants.to_numpy()

replicates = 5

# real concs in wells. Stocks in ug/ml diluted by 20x
calibrant_range = ["0","500","750","1000", "1250", "1500", "1750","2000"]

#calibrants_np_2d = np.reshape(calibrants_np, (int(len(calibrants_np)/replicates), replicates))

import seaborn as sns

sns.set()
ax1 = sns.heatmap(calibrants_np, annot=True, cmap="coolwarm")
ax1.set_ylabel("BSA ug/Ml")
ax1.set_title("Calibrant Raw Absorbances.")
ax1.set_xlabel("Replicate #")
plt.yticks(np.arange(8)+0.5,calibrant_range, rotation=0, fontsize="10")

plt.savefig("output/BSA_absorbances_heat.png")

plt.show()

In [ ]:
calibrants_df = pd.DataFrame(calibrants_np, columns=["Rep1", "Rep2", "Rep3", "Rep4", "Rep5"], index=calibrant_range)

calibrants_df["Mean"] = calibrants_df.mean(axis=1)
calibrants_df["SEM"] = calibrants_df.iloc[:,:3].sem(axis=1)

calibrants_df_avg = calibrants_df[["Mean", "SEM"]]

#plot 

x = calibrants_df_avg.index
y = calibrants_df_avg['Mean']

plt.scatter(x,y)
plt.errorbar(x,y, yerr=calibrants_df_avg['SEM'], fmt="|", color="r")

plt.title("Absorbance of BSA Standards")
plt.ylabel("Absorbance")
plt.xlabel("BSA ug/Ml")
plt.savefig("output/BSA_Standard_Absorbances_scatter.png")
plt.show()


In [2]:

# calibrant concs
calibrant_range_numerical = [float(ele) for ele in calibrant_range]
print(calibrant_range_numerical)

# fit curves
#function for looping?
model = LinearRegression()
# fitc data in to array and reverse the order
x = np.array(calibrants_df_avg["Mean"]).reshape(-1,1)

# List of nM concs into array
y = np.array(calibrant_range_numerical).reshape(-1,1)

x = x.reshape(-1,1)

# Fit
model.fit(x,y)
r_sq = model.score(x, y)
print('coefficient of determination:', r_sq)

#Plot
plt.scatter(x, y, color='g')
plt.plot(x, model.predict(x), color='k')

textstr ='r²: ' +str(round(r_sq,2))
plt.text(calibrants_df_avg["Mean"].min(), calibrant_range_numerical[-1], textstr, color='r', fontsize=10)

plt.title('Standard Curve')
plt.xlabel('Arbitrary Absorbance Units')
plt.ylabel('BSA μg/Ml')
plt.savefig("output/BSA_Standard_Curve.png")
plt.show()




NameError: name 'calibrant_range' is not defined

In [3]:
# samples mean & sem

replicates = 3

sample_data = corrected_data.iloc[:3,5:8]

#new_low = corrected_data.iloc[0,:].copy()
#new_high = corrected_data.iloc[2,:].copy()

#sample_data.iloc[0,:] = new_high
#sample_data.iloc[2,:] = new_low


sample_data = sample_data.values

sample_list =["20x","40x","100x"]

sample_data = pd.DataFrame(sample_data, columns=["Rep1", "Rep2", "Rep3"], index = sample_list).astype(float)
sns.set()


sns.heatmap(sample_data, annot=True, cmap="coolwarm")



plt.ylabel("Dilution Factor")
plt.title("Absorbances of Diluted samples")
plt.xlabel("Replicate #")
plt.yticks(np.arange(sample_data.shape[0])+0.5, sample_list, rotation=0, fontsize="10")
plt.savefig("output/Sample_Absorbances_heat.png")



NameError: name 'corrected_data' is not defined

In [4]:


sample_data["Mean"] = sample_data.mean(axis=1)
sample_data["SEM"] = sample_data.iloc[:,:3].sem(axis=1)


samples_df_avg = sample_data[["Mean", "SEM"]]

ax3 = samples_df_avg.plot(kind="bar", yerr="SEM", legend=False)


ax3.set_title("Mean Absorbances of Sample Dilutions")
ax3.set_ylabel("Absorbance")
ax3.set_xlabel("Dilution Factor")
plt.xticks(np.arange(samples_df_avg.shape[0]),sample_list, rotation=0, fontsize="10")
plt.savefig("output/Sample_Absorbances_Bar.png")

NameError: name 'sample_data' is not defined

In [5]:
#### subset samples that don't fall in linear range
acceptable_samples = sample_data


#get the index again for the names
acceptable_samples_list = acceptable_samples.index
print(acceptable_samples)

NameError: name 'sample_data' is not defined

In [ ]:
# convert samples
predicted_ug_per_ml = model.predict(np.array(acceptable_samples.iloc[:,:3]).reshape(-1,1))
predicted_ug_per_ml = np.reshape(predicted_ug_per_ml,(int(len(predicted_ug_per_ml)/replicates), replicates))
predicted_ug_per_ml = pd.DataFrame(predicted_ug_per_ml, columns=["Rep1", "Rep2", "Rep3"], index = acceptable_samples_list)

print(predicted_ug_per_ml)

In [ ]:
#account for the dilutions
numrows = predicted_ug_per_ml.shape[0]

print("Raw dataset:")
print(predicted_ug_per_ml)
print(" ")

for index, row in predicted_ug_per_ml.iterrows():
    # get the dilution factor from the index
    ind = str(index)
    ind = ind[:-1]
    ind = int(ind)

    print("Multiplying row " + str(index) +  " by dilution factor: "+ str(ind))
    
    predicted_ug_per_ml.loc[index, "Rep1"] = predicted_ug_per_ml.loc[index, "Rep1"]*ind
    predicted_ug_per_ml.loc[index, "Rep2"] = predicted_ug_per_ml.loc[index, "Rep2"]*ind
    predicted_ug_per_ml.loc[index, "Rep3"] = predicted_ug_per_ml.loc[index, "Rep3"]*ind

# get list of row indexes

print(" ")
print("dataset multipled by dilution factor")
print(" ")


print(predicted_ug_per_ml)

# Use that to perform that calculations just on the right columns
# recalculate mean and SEM

predicted_ug_per_ml['ug/ml'] = predicted_ug_per_ml.loc[list(predicted_ug_per_ml.index),["Rep1", "Rep2","Rep3"]].mean(axis=1)
predicted_ug_per_ml['ug/ml SEM'] = predicted_ug_per_ml.loc[list(predicted_ug_per_ml.index),["Rep1", "Rep2","Rep3"]].sem(axis=1)

print(" ")
print("dataset in ug/ml")
print(" ")

print(predicted_ug_per_ml[['ug/ml','ug/ml SEM']])


In [ ]:

#get index to use and names
predicted_ug_per_ml = predicted_ug_per_ml.reset_index()
#convert all to strings
predicted_ug_per_ml['index'] = predicted_ug_per_ml['index'].astype(str)
print(predicted_ug_per_ml)



In [ ]:
######################### convert ug/ml to mg/ml
# grab the raw values
predicted_mg_per_ml = predicted_ug_per_ml[['Rep1','Rep2','Rep3']]
# convert
predicted_mg_per_ml = predicted_mg_per_ml/1000
# append index
predicted_mg_per_ml['index'] = predicted_ug_per_ml['index']


print(predicted_mg_per_ml)

In [ ]:
####################### generate means and stats
predicted_mg_per_ml['mg/ml'] = predicted_mg_per_ml.loc[list(predicted_mg_per_ml.index),["Rep1", "Rep2","Rep3"]].mean(axis=1)
predicted_mg_per_ml['mg/ml SEM'] = predicted_mg_per_ml.loc[list(predicted_mg_per_ml.index),["Rep1", "Rep2","Rep3"]].sem(axis=1)

print(predicted_mg_per_ml)

In [ ]:


samples_bar_av_df = predicted_mg_per_ml[["mg/ml", "mg/ml SEM"]]
print(samples_bar_av_df)

ax4 = samples_bar_av_df.plot(kind="bar", yerr="mg/ml SEM", legend=False)
ax4.set_title("Samples Mean Concentrations & SEMs")
ax4.set_ylabel("Concentration in Mg/Ml")
ax4.set_xlabel("Dilution Factor")
plt.xticks(np.arange(samples_bar_av_df.shape[0]),acceptable_samples_list, rotation=0, fontsize="10")
plt.savefig("output/Conc_bar.png")




In [ ]:
############### check is the SEM on the mean of means is too large.


#grab all of the replicates
checker_set_df = predicted_mg_per_ml[['Rep1', 'Rep2', 'Rep3']]

# initalise an empty series
checker_set_series = pd.Series(dtype='float64')

# go through the replicates and put them in the series
for name, values in checker_set_df.iteritems():
    checker_set_series = checker_set_series.append(values)

# reset the series
checker_set_series = checker_set_series.reset_index(drop=True)
print('')
#print('Compiled Replicates')
#print(checker_set_series)
print('')

# generate the mean and sem
checker_mean, checker_sem = (checker_set_series.mean(),checker_set_series.sem())

checker_mean_5_percent = checker_mean/20

print('Mean of means: ')
print(checker_mean)
print(' ')
print('SEM of means: ')
print(checker_sem)
print(' ')
print('Is the SEM greater than 5% of the Mean?:')
print(checker_sem > checker_mean_5_percent)

In [ ]:
############## Is the sem greater than 5% of the mean?
# If so: check for outliers.
if checker_sem > checker_mean_5_percent:
    
    ####################################################################################################################
    ####################################################################################################################
    ############### Start if Greater than BlOCK
    
    print('SEM is greater and outliers MIGHT have been identified')
    
    
    #################### box plot to identify outlier
    plt.boxplot(x=predicted_mg_per_ml['mg/ml'], vert=False)
    plt.title("Box Plot of Mean Concentrations")
    plt.xlabel("Mg/Ml")

    plt.savefig("output/Box_plot.png")
    plt.show()
    
    
    #########################################################
    #calculate interquartile range of values in the 'points' column
    q75, q25 = np.percentile(predicted_mg_per_ml['mg/ml'], [75 ,25])
    
    print(' ')
    print('q25: '+str(q25))
    print(' ')
    print('Mean: '+str(checker_mean))
    print(' ')
    print('q75: '+str(q75))
    print(' ')    
        
    ######### retain values that are within the interquartile range
    acceptable_concs = predicted_mg_per_ml.loc[(predicted_mg_per_ml["mg/ml"] >=q25) & (predicted_mg_per_ml["mg/ml"] <=q75)]

    print('The concentrations that are retained after outlier filter')
    print(' ')
    print(acceptable_concs)
    print(' ')
    

    #########################################################
    ####   plot a bar chart of the accepted concentrations
    
    barplot_df = acceptable_concs[['index','mg/ml','mg/ml SEM']]
    ax4 = barplot_df.plot(kind="bar", yerr="mg/ml SEM", legend=False)
    ax4.set_title("Samples Mean Concentrations & SEMs")
    ax4.set_ylabel("Concentration in Mg/Ml")
    ax4.set_xlabel("Dilution Factor")
    plt.xticks(np.arange(acceptable_concs.shape[0]),acceptable_concs['index'], rotation=0, fontsize="10")
    plt.savefig("output/Best_concs_bar.png")
    
    #############################################################
    ##### using the accepted concentrations to generate a consensus concentration and export it.

    subsetted = acceptable_concs[['Rep1', 'Rep2', 'Rep3']]

    compiled = pd.Series(dtype='float64')

    for name, values in subsetted.iteritems():
        compiled = compiled.append(values)

    compiled = compiled.reset_index(drop=True)
    print('')
    print('Compiled Replicates')
    print(compiled)
    print('')
    consensus_mean, consensus_sem = (compiled.mean(),compiled.sem())
    print('Generated Consensus Concentration:')
    print('consensus_mean: '+ str(consensus_mean))
    print('consensus_sem: ' + str(consensus_sem))
    consensus_num_samples = acceptable_concs.shape[0]
    print('Generated from # dilutions: ' + str(consensus_num_samples))
    
    print(' ')
    print('Saving to Consensus_concentration.csv ...')
    consensus_printout = pd.DataFrame(data = [[consensus_mean, consensus_sem, consensus_num_samples]], columns=['Consensus_Mean','Consensus_Sem','Calculated_From_#_Dilutions'])
    consensus_printout.to_csv('output/Consensus_concentration.csv')
    
    ############## END OF IF Greater than BLOCK
    ###################################################################################################################################
    ###################################################################################################################################

else:
    
    ####################################################################################################################################
    ####################################################################################################################################
    ############# Start of IF not Greater BLOCK
    
    print('SEM is not greater and no outliers have been identified')
    
    subsetted = predicted_mg_per_ml[['Rep1', 'Rep2', 'Rep3']]

    compiled = pd.Series(dtype='float64')

    for name, values in subsetted.iteritems():
        compiled = compiled.append(values)

    compiled = compiled.reset_index(drop=True)
    print('')
    print('Compiled Replicates')
    print(compiled)
    print('')
    consensus_mean, consensus_sem = (compiled.mean(),compiled.sem())
    print('Generated Consensus Concentration:')
    print('consensus_mean: '+ str(consensus_mean))
    print('consensus_sem: ' + str(consensus_sem))
    consensus_num_samples = predicted_mg_per_ml.shape[0]
    print('Generated from # dilutions: ' + str(consensus_num_samples))
    
    print(' ')
    print('Saving to Consensus_concentration.csv ...')
    consensus_printout = pd.DataFrame(data = [[consensus_mean, consensus_sem, consensus_num_samples]], columns=['Consensus_Mean','Consensus_Sem','Calculated_From_#_Dilutions'])
    consensus_printout.to_csv('Consensus_concentration.csv')


    